In [ ]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-_x290a4q
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-_x290a4q
  Resolved https://github.com/huggingface/transformers to commit 52daf4ec768fb9ffe84a0c373834172a7c54aecc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9800639 sha256=cf211927cb08cb70786f4d08587977d90ba6d6eb86bcad4f0e4ab9137b0e99c6
  Stored in directory: /tmp/pip-ephem-wheel-cache-wwfdpia4/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [ ]:
!pip install qwen_vl_utils
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
model = Qwen2VLForConditionalGeneration.from_pretrained(
"Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto")
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct-GPTQ-Int4")
from PIL import Image

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
image=Image.open(r'test3.jpg')
query="what time of the day is it?"
print(prompt(image,query))

The handwriting in the image appears to be in a language that uses the Latin alphabet, which is commonly used in English-speaking countries. The text reads: "We want to complete SIH and win the competition. ABCDEFGHIJKLMNOPQRSTUVWXYZ." 

If you are asking about the time of the day, it is not possible to determine the time from the handwriting alone.


In [ ]:
# Initialize an empty list to store the conversation history
conversation_history = []

def prompt(img, query):
    # Append the new user input to the conversation history
    conversation_history.append(
        {
            "role": "user",
            "content": [
                {"type": "image", "image": img},
                {"type": "text", "text": query},

            ],
        }
    )

    # Prepare the messages with the full conversation history
    messages = conversation_history[:]

    # Process the messages
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to('cuda')

    # Inference: Generation of the output
    generated_ids = model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    # Append the model's response to the conversation history
    conversation_history.append(
        {
            "role": "system",
            "content": {"type": "text", "text": output_text[0]},
        }
    )

    return output_text[0]


In [ ]:
def prompt_text(query):
    # Append the new user input to the conversation history
    conversation_history.append(
        {
            "role": "user",
            "content": [
                {"type": "text", "text": query},

            ],
        }
    )

    # Prepare the messages with the full conversation history
    messages = conversation_history[:]

    # Process the messages
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to('cuda')

    # Inference: Generation of the output
    generated_ids = model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    # Append the model's response to the conversation history
    conversation_history.append(
        {
            "role": "system",
            "content": {"type": "text", "text": output_text[0]},
        }
    )

    return output_text[0]

In [ ]:
print(prompt_text("give me just the text extracted"))

We want to complete SIH and win the competition. ABCDEFGHIJKLMNOPQRSTUVWXYZ.


In [ ]:
conversation_history

[{'role': 'user',
  'content': [{'type': 'image',
    'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2269x1276>},
   {'type': 'text', 'text': 'what time of the day is it?'}]},
 {'role': 'system',
  'content': {'type': 'text',
   'text': 'The handwriting in the image appears to be in a language that uses the Latin alphabet, which is commonly used in English-speaking countries. The text reads: "We want to complete SIH and win the competition. ABCDEFGHIJKLMNOPQRSTUVWXYZ." \n\nIf you are asking about the time of the day, it is not possible to determine the time from the handwriting alone.'}},
 {'role': 'user',
  'content': [{'type': 'text',
    'text': 'okay is this in english language and what does that text mean'}]},
 {'role': 'system',
  'content': {'type': 'text',
   'text': 'The text in the image is written in a language that appears to be English. The text reads:\n\n"We want to complete SIH and win the competition. ABCDEFGHIJKLMNOPQRSTUVWXYZ."\n\nThis text seems to b